In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
from utils import generate_table, Cell, to_latex, merge_cols

In [5]:
data = pd.read_csv("../../results/cmnist/3_digits/first_decent_results.csv")

In [6]:
data["Group"].value_counts()

.same_as_for2.zs2.even_longer.3dig_4miss    30
Name: Group, dtype: int64

In [7]:
data = data.replace({"Group": {
    ".same_as_for2.zs2.even_longer.3dig_4miss": "no cluster",
}}, inplace=False)

In [10]:
data.columns

Index(['Name', 'misc.seed', 'Runtime', 'Group', 'Hostname', 'State',
       'bias.log_dataset', 'fdm.balanced_context', 'fdm.lr',
       'misc.data_split_seed', 'misc.exp_group', 'misc.log_method',
       'Accuracy (pytorch_classifier)', 'Accuracy Discriminator (zy)',
       'Accuracy Predictor s', 'Accuracy Predictor y',
       'Accuracy_colour_0.0 (pytorch_classifier)',
       'Accuracy_colour_0.0-colour_1.0 (pytorch_classifier)',
       'Accuracy_colour_0.0-colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_0.0÷colour_1.0 (pytorch_classifier)',
       'Accuracy_colour_0.0÷colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_1.0 (pytorch_classifier)',
       'Accuracy_colour_1.0-colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_1.0÷colour_2.0 (pytorch_classifier)',
       'Accuracy_colour_2.0 (pytorch_classifier)', 'ELBO', 'Learning rate',
       'Loss Adversarial', 'Loss Generator', 'Loss Predictor s',
       'Loss Predictor y', 'Loss reconstruction', 'Prior Loss

In [13]:
cols = {
#     'Clust/Context Accuracy': "Clust Acc",
#     'Clust/Context NMI': "Clust NMI",
#     'Clust/Context ARI': "Clust ARI",
    "Accuracy (pytorch_classifier)": "Accuracy",
    'prob_pos_colour_0.0÷colour_1.0 (pytorch_classifier)': "AR ratio",
    "Renyi preds and s (pytorch_classifier)": "Renyi corr",
    "TPR_colour_0.0÷colour_1.0 (pytorch_classifier)": "TPR ratio",
    'TNR_colour_0.0÷colour_1.0 (pytorch_classifier)': "TNR ratio",
}

In [17]:
groupby = "Group"
res = (data[[groupby] + list(cols)]
        .groupby([groupby])
        .agg(Cell(round_to=3)).rename(columns=cols, inplace=False)
        .reset_index(level=[groupby], inplace=False)
        .rename(columns={groupby: "Method"}, inplace=False)
       )
res

,Method,Accuracy,AR ratio,Renyi corr,TPR ratio,TNR ratio
0,no cluster,0.882 $\pm$ 0.076,0.7 $\pm$ 0.213,0.264 $\pm$ 0.169,0.993 $\pm$ 0.009,0.703 $\pm$ 0.292


In [19]:
# baseline files
baseline_files = {
    "CNN": "cmnist_cnn_baseline_color_60epochs.csv",
}

In [20]:
def collate(file_dict, dir_):
    df_all = pd.DataFrame()
    for log_method, filename in file_dict.items():
        df = pd.read_csv(Path("../../results/cmnist/3_digits") / dir_ / filename)
#         df.insert(0, "exp_group", exp_group)
        df.insert(0, "log_method", log_method)
        if log_method == "DRO":
            df["log_method"] = df["log_method"] + " (" + df["eta"].astype(str) + ")"
        df_all = pd.concat([df_all, df], axis="index", ignore_index=True, sort=False)
    return df_all

In [21]:
baselines = collate(baseline_files, ".")

In [22]:
baselines.columns

Index(['log_method', 'seed', 'data', 'method', 'wandb_url', 'Accuracy', 'TPR',
       'TNR', 'Renyi preds and s', 'Accuracy_colour_2.0',
       'Accuracy_colour_1.0', 'Accuracy_colour_0.0',
       'Accuracy_colour_0.0-colour_1.0', 'Accuracy_colour_0.0-colour_2.0',
       'Accuracy_colour_1.0-colour_2.0', 'Accuracy_colour_0.0÷colour_1.0',
       'Accuracy_colour_0.0÷colour_2.0', 'Accuracy_colour_1.0÷colour_2.0',
       'prob_pos_colour_2.0', 'prob_pos_colour_1.0', 'prob_pos_colour_0.0',
       'prob_pos_colour_0.0-colour_1.0', 'prob_pos_colour_0.0-colour_2.0',
       'prob_pos_colour_1.0-colour_2.0', 'prob_pos_colour_0.0÷colour_1.0',
       'prob_pos_colour_0.0÷colour_2.0', 'prob_pos_colour_1.0÷colour_2.0',
       'TPR_colour_2.0', 'TPR_colour_1.0', 'TPR_colour_0.0',
       'TPR_colour_0.0-colour_1.0', 'TPR_colour_0.0-colour_2.0',
       'TPR_colour_1.0-colour_2.0', 'TPR_colour_0.0÷colour_1.0',
       'TPR_colour_0.0÷colour_2.0', 'TPR_colour_1.0÷colour_2.0',
       'TNR_colour_2.0', 'TN

In [24]:
bl_cols = {
#     'Clust/Context Accuracy': "Clust Acc",
#     'Clust/Context NMI': "Clust NMI",
#     'Clust/Context ARI': "Clust ARI",
    "Accuracy": "Accuracy",
    'prob_pos_colour_0.0÷colour_1.0': "AR ratio",
    "Renyi preds and s": "Renyi corr",
    "TPR_colour_0.0÷colour_1.0": "TPR ratio",
    'TNR_colour_0.0÷colour_1.0': "TNR ratio",
}

In [27]:
groupby = "log_method"
res2 = (baselines[[groupby] + list(bl_cols)]
        .groupby([groupby])
        .agg(Cell(round_to=3)).rename(columns=bl_cols, inplace=False)
        .reset_index(level=[groupby], inplace=False)
        .rename(columns={groupby: "Method"}, inplace=False)
       )
res2

,Method,Accuracy,AR ratio,Renyi corr,TPR ratio,TNR ratio
0,CNN,0.627 $\pm$ 0.054,0.338 $\pm$ 0.024,0.784 $\pm$ 0.069,0.996 $\pm$ 0.003,0.039 $\pm$ 0.054


In [29]:
table = pd.concat([res, res2]).reset_index(drop=True)
table

,Method,Accuracy,AR ratio,Renyi corr,TPR ratio,TNR ratio
0,no cluster,0.882 $\pm$ 0.076,0.7 $\pm$ 0.213,0.264 $\pm$ 0.169,0.993 $\pm$ 0.009,0.703 $\pm$ 0.292
1,CNN,0.627 $\pm$ 0.054,0.338 $\pm$ 0.024,0.784 $\pm$ 0.069,0.996 $\pm$ 0.003,0.039 $\pm$ 0.054


In [30]:
print(table.to_latex(index=False, escape=False))

\begin{tabular}{llllll}
\toprule
     Method &           Accuracy &           AR ratio &         Renyi corr &          TPR ratio &          TNR ratio \\
\midrule
 no cluster &  0.882 $\pm$ 0.076 &    0.7 $\pm$ 0.213 &  0.264 $\pm$ 0.169 &  0.993 $\pm$ 0.009 &  0.703 $\pm$ 0.292 \\
        CNN &  0.627 $\pm$ 0.054 &  0.338 $\pm$ 0.024 &  0.784 $\pm$ 0.069 &  0.996 $\pm$ 0.003 &  0.039 $\pm$ 0.054 \\
\bottomrule
\end{tabular}

